In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
import pyreadstat
import os
#from ydata_profiling import ProfileReport
import datetime as dt
import matplotlib.pyplot as plt

%matplotlib inline

pd.options.display.max_columns = None
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [4]:
cwd = os.getcwd()
print(cwd)
filepath = cwd+r"/data/prams_standard_phase8.sas7bdat"
print(filepath)

/Users/ndeen/git/paidleave_mh
/Users/ndeen/git/paidleave_mh/data/prams_standard_phase8.sas7bdat


#### Data Exploration

Standard

In [5]:
# import the dataset
df_s, meta = pyreadstat.read_sas7bdat(filepath)

In [6]:
# profile the dataset
#s_profile = ProfileReport(df_s, title="Profiling Report", minimal=True)
#s_profile.to_file("profiles/s_profile.html")

**Relevant lit**:

* Analysis of brief screening tools for the detection of postpartum depression: Comparisons of the PRAMS 6-item instrument, PHQ-9, and structured interviews (https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3722286/)

* "Postpartum depression was defined as responses of “often” or “always” to two questions: “Since your baby was born, how often have you felt down, depressed, or hopeless?” and “Since your new baby was born, how often have you had little interest or little pleasure in doing things?”." (https://www.nmhealth.org/data/view/maternal/2436/)


In [7]:
# identify variables of interest
voi = [
    'ID',
    'HISPANIC',
    'STATE',
    'MAT_RACE_PU',
    'YY4_DOB',
    'MH_PPDPR',
    'MH_PPINT',
]

In [8]:
df_s = df_s[voi]
df_s.head()

,ID,HISPANIC,STATE,MAT_RACE_PU,YY4_DOB,MH_PPDPR,MH_PPINT
0,2016AK327002,1.0,AK,NaN,2016.0,5.0,2.0
1,2016AK327006,1.0,AK,NaN,2016.0,3.0,4.0
2,2016AK327007,1.0,AK,NaN,2016.0,3.0,3.0
3,2016AK327009,1.0,AK,NaN,2016.0,5.0,5.0
4,2016AK327010,1.0,AK,NaN,2016.0,1.0,1.0


In [9]:
# recode variables for depression

# Define the conditions for postpartum depression
depression_conditions = ((df_s['MH_PPDPR'] == 1) | (df_s['MH_PPDPR'] == 2)) & ((df_s['MH_PPINT'] == 1) | (df_s['MH_PPINT'] == 2))

# Recode the postpartum depression variable
df_s['ppd'] = depression_conditions.astype(int)

In [10]:
df_s.head()

,ID,HISPANIC,STATE,MAT_RACE_PU,YY4_DOB,MH_PPDPR,MH_PPINT,ppd
0,2016AK327002,1.0,AK,NaN,2016.0,5.0,2.0,0
1,2016AK327006,1.0,AK,NaN,2016.0,3.0,4.0,0
2,2016AK327007,1.0,AK,NaN,2016.0,3.0,3.0,0
3,2016AK327009,1.0,AK,NaN,2016.0,5.0,5.0,0
4,2016AK327010,1.0,AK,NaN,2016.0,1.0,1.0,1


In [52]:
# ppd by mwternal race (1=YES 0=NO)
df_s[['ID','ppd']].groupby(['ppd'])['ID'].nunique()

ppd
0    212245
1      9136
Name: ID, dtype: int64

Occupational Status

Maternity Leave
 - C4. At any time during your most recent pregnancy, did you work at a job for pay?
 - C7. Have you returned to the job you had during your most recent pregnancy? Check ONE answer

NOTE: C8 requires C7 and C4. If a site adds a site-specific option to C8, insert “I took…” for options such as Family Medical Leave and “I took leave and used…” for options such as Temporary/Short-term Disability Insurance.

- C8. Did you take leave from work after your new baby was born?
- C9. How did you feel about the amount of time you were able to take off after the birth of your new baby?
- C10. Did any of the following things affect your decision about taking leave from work after your new baby was born?
- C14. How many weeks or months of leave, in total, did you take or will you take?

In [21]:
filepath = r"/Users/ndeen/git/paidleave_mh/data/phase8_2016_2021_std_c.sas7bdat"
df_c, meta = pyreadstat.read_sas7bdat(filepath)

In [12]:
# profile the dataset
#c_profile = ProfileReport(df_c, title="Profiling Report", minimal=True)
#c_profile.to_file("profiles/c_profile.html")

In [22]:
df_c.head()

,ID,WRK_PREG,WRK_TITL_RAW,WRK_DUTY_RAW,WRKRETRN_RAW,WRKPDLV_RAW,WRKUPDLV_RAW,WRKNOLV_RAW,LVAFFORD_RAW,LVAFRAID_RAW,LVWORKLD_RAW,LVUNPAID_RAW,LVNOFLEX_RAW,LVENOUGH_RAW,WRK_TITL,WRK_DUTY,WRKRETRN,WRKPDLV,WRKUPDLV,WRKNOLV,LVAFFORD,LVAFRAID,LVWORKLD,LVUNPAID,LVNOFLEX,LVENOUGH,C_WRKSCH_RAW,CC_WHO6_RAW,CC_OTH6_RAW,CC_FEEL_RAW,LV_AMTU_RAW,LV_AMT_RAW,WRKFEEL_RAW,C_WRKSCH,CC_OTH6,CC_WHO6,CC_FEEL,LV_AMTU,LV_AMT,WRKFEEL,WRKFMLV_RAW,WRKFMLV,WRK_TYPE_RAW,WRK_IDK_RAW,WRK_TYPE,WRK_IDK,WRKYCTDI_RAW,WRKYCTDI,DADLEAVE,DADLEAVE_DK,WRKSCHED_RAW,WRKRETRN_C6_RAW,WRKPDLV_C6_RAW,WRKUPDLV_C6_RAW,WRKNOLV_C6_RAW,LV_AMTU_C6_RAW,LV_AMT_C6_RAW,LVAFFORD_C6_RAW,LVAFRAID_C6_RAW,LVWORKLD_C6_RAW,LVUNPAID_C6_RAW,LVNOFLEX_C6_RAW,LVENOUGH_C6_RAW,WRKSCHED,WRKRETRN_C6,WRKPDLV_C6,WRKUPDLV_C6,WRKNOLV_C6,LV_AMTU_C6,LV_AMT_C6,LVAFFORD_C6,LVAFRAID_C6,LVWORKLD_C6,LVUNPAID_C6,LVNOFLEX_C6,LVENOUGH_C6
0,2016LA232001,2.0,CMA/smoke monitor,Made sure patients was wearing smoke smocks an...,2.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,CMA/smoke monitor,Made sure patients was wearing smoke smocks an...,2.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2016LA232002,2.0,Teacher,High school basic courses,3.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,Teacher,High school basic courses,3.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2016LA232003,2.0,Accountant,"Typing, research on computer, auditing, payrol...",3.0,2.0,2.0,1.0,2.0,1.0,1.0,1.0,1.0,2.0,Accountant,"Typing, research on computer, auditing, payrol...",3.0,2.0,2.0,1.0,2.0,1.0,1.0,1.0,1.0,2.0,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2016LA232004,1.0,,,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2016LA232006,2.0,Teller,Money transactions lifting 25 lbs. or more daily,3.0,2.0,1.0,1.0,2.0,2.0,1.0,2.0,2.0,2.0,Teller,Money transactions lifting 25 lbs. or more daily,3.0,2.0,1.0,1.0,2.0,2.0,1.0,2.0,2.0,2.0,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
# c8 - did you take leave from work after your new baby was born?
df_c = df_c[[
     'ID',
     'WRKPDLV', # paid
     'WRKUPDLV', # unpaid
     'WRKNOLV' # no leave
]]

In [24]:
# paid leave (1=NO, 2=YES)
df_c[['ID','WRKPDLV']].groupby(['WRKPDLV'])['ID'].nunique()

WRKPDLV
1.0    18186
2.0    17984
Name: ID, dtype: int64

In [25]:
# unpaid leave (1=NO, 2=YES)
df_c[['ID','WRKUPDLV']].groupby(['WRKUPDLV'])['ID'].nunique()

WRKUPDLV
1.0    16430
2.0    19736
Name: ID, dtype: int64

In [26]:
# no leave (1=NO, 2=YES)
df_c[['ID','WRKNOLV']].groupby(['WRKNOLV'])['ID'].nunique()

WRKNOLV
1.0    34495
2.0     1642
Name: ID, dtype: int64

In [40]:
# merge standard and occupational status dataframes 
merged_df = pd.merge(df_s, df_c, on='ID', how='outer')
merged_df = merged_df.drop(columns=['MH_PPDPR', 'MH_PPINT'])

In [41]:
# number of rows without any missing 
len(merged_df.dropna())

32113

In [42]:
cohort = merged_df.dropna()

In [43]:
cohort.head()

,ID,HISPANIC,STATE,MAT_RACE_PU,YY4_DOB,ppd,WRKPDLV,WRKUPDLV,WRKNOLV
8075,2016LA232001,1.0,LA,3.0,2016.0,0,1.0,2.0,1.0
8076,2016LA232002,1.0,LA,3.0,2016.0,0,1.0,2.0,1.0
8077,2016LA232003,1.0,LA,3.0,2016.0,0,2.0,2.0,1.0
8079,2016LA232006,1.0,LA,3.0,2016.0,0,2.0,1.0,1.0
8080,2016LA232007,1.0,LA,3.0,2016.0,0,2.0,2.0,1.0


In [44]:
cohort['STATE'].unique()

array(['LA', 'MA', 'MD', 'MO', 'NH', 'NM', 'NY', 'WI', 'YC', 'OR', 'TN'],
      dtype=object)

In [46]:
cohort['YY4_DOB'].unique()

array([2016., 2017., 2018., 2019., 2020., 2021.])

In [49]:
# sample size with the treatment (paid leave, 1=NO, 2=YES)
cohort.groupby(['WRKPDLV'])['ID'].nunique()

WRKPDLV
1.0    16237
2.0    15876
Name: ID, dtype: int64

In [50]:
# sample size with the treatment (unpaid leave, 1=NO, 2=YES)
cohort.groupby(['WRKUPDLV'])['ID'].nunique()

WRKUPDLV
1.0    14739
2.0    17374
Name: ID, dtype: int64

In [51]:
# sample size with the treatment (no leave, 1=NO, 2=YES)
cohort.groupby(['WRKNOLV'])['ID'].nunique()

WRKNOLV
1.0    30612
2.0     1501
Name: ID, dtype: int64

Mental Health

In [17]:
filepath = r"C:\Users\Nigel\OneDrive\1_GradSchool\4_Dissertation\1_paidleave_mhdisparities\PRAMS  Phase 8 Standard May 22 2024\Mental Health\phase8_2016_2021_std_m.sas7bdat"
df_m, meta = pyreadstat.read_sas7bdat(filepath)

In [18]:
# profile the dataset
m_profile = ProfileReport(df_m, title="Profiling Report", minimal=True)
m_profile.to_file("profiles/m_profile.html")

Export report to file: 100%|██████████| 1/1 [00:00<00:00, 499.74it/s]


In [19]:
df_m.head()

,ID,MH_PPDX,MH_PPRX_RAW,MH_PPRX,MH_PPHLP,MH_PPPSY_RAW,MH_PPPSY,PP_PANIC,PP_NREST,MH_PGRX8_RAW,MH_PGRX8,MH_PGHP8_RAW,MH_PGHP8,MH_HDANX,MH_RXANX_RAW,MH_RXANX,DEPR_TLK,MH_PGPSY8_RAW,MH_PREG,MH_PGPSY8,MH_ANXHP,MH_PGANX,MH_ANX,MH_ANXCN_RAW,MH_ANXCN
0,2016CO224001,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2016CO224003,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2016CO224006,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2016CO224007,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2016CO224009,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
df_m = df_m[[
     'ID',
     'MH_PPDX', # M5. Since your new baby was born, has a healthcare provider told you that you had depression? 1=NO, 2=YES
     'MH_ANX', # M15. Since your new baby was born, has a healthcare provider told you that you had anxiety? 1=NO, 2=YES
]]

In [21]:
# depression
df_m[['ID','MH_PPDX']].groupby(['MH_PPDX'])['ID'].nunique()

MH_PPDX
1.0    34573
2.0     4232
Name: ID, dtype: int64

In [22]:
print("Percent missing a depression answer: {}".format(round(df_m['MH_PPDX'].isnull().sum() / len(df_m) * 100,2)))

Percent missing a depression answer: 52.55


In [23]:
# anxiety
df_m[['ID','MH_ANX']].groupby(['MH_ANX'])['ID'].nunique()

MH_ANX
1.0    718
2.0     69
Name: ID, dtype: int64

In [24]:
print("Percent missing an anxiety answer: {}".format(round(df_m['MH_ANX'].isnull().sum() / len(df_m) * 100,2)))

Percent missing an anxiety answer: 99.04


In [30]:
df_m = df_m[[
     'ID',
     'MH_PPDX', # M5. Since your new baby was born, has a healthcare provider told you that you had depression? 1=NO, 2=YES
]]

Merge to see the crosstabs b/w leave status and depression diagnoses

In [31]:
# First, merge df_s and df_m on 'ID'
merged_df = pd.merge(df_s, df_m, on='ID', how='outer')

In [32]:
# Then, merge the resulting DataFrame with df_c on 'ID'
merged_df = pd.merge(merged_df, df_c, on='ID', how='outer')

In [34]:
# Drop rows with any NaN values
cleaned_df = merged_df.dropna()
cleaned_df.head()


,ID,HISPANIC,STATE,MAT_RACE_PU,YY4_DOB,MH_PPDX,WRKPDLV,WRKUPDLV,WRKNOLV
16417,2016NH038002,1.0,NH,2.0,2016.0,1.0,1.0,2.0,1.0
16418,2016NH038003,1.0,NH,2.0,2016.0,1.0,2.0,1.0,1.0
16419,2016NH038005,1.0,NH,2.0,2016.0,1.0,1.0,2.0,1.0
16420,2016NH038006,1.0,NH,2.0,2016.0,2.0,1.0,2.0,1.0
16421,2016NH038007,1.0,NH,2.0,2016.0,1.0,2.0,1.0,1.0


In [35]:
len(cleaned_df)

7221

In [36]:
cleaned_df[['ID','STATE']].groupby('STATE')['ID'].nunique()

STATE
NH    1911
NY    1504
YC    3806
Name: ID, dtype: int64

Figure out how to increase the sample (perhaps look at combining categories or removing columns to have fewer null values Ex. combine no leave with unpaid leave? - how have others done this)